# 네이버 영화평 감성 분석

In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

- 데이터 가져오기: '네이버 영화리뷰 데이터'로 검색

In [8]:
train_df = pd.read_csv('https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt', sep='\t')
test_df = pd.read_csv('https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt', sep='\t')
train_df.shape, test_df.shape 

((150000, 3), (50000, 3))

In [3]:
train_df.head(3)

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0


### 1. 데이터 전처리

- 트레인 데이터 셋

In [9]:
# Null 데이터가 있는지 확인
train_df.document.isna().sum()

5

In [12]:
# Null 데이터 제거
train_df.dropna(how='any', inplace=True)
train_df.shape

(149995, 3)

In [13]:
# 중복 여부 확인
train_df.document.nunique()

146182

In [14]:
# 중복 데이터 제거
train_df.drop_duplicates(subset=['document'], inplace=True)
train_df.shape

(146182, 3)

In [15]:
# 긍정/부정 데이터 분포
train_df.label.value_counts()

0    73342
1    72840
Name: label, dtype: int64

- 테스트 데이터 셋

In [16]:
# Null 데이터 확인
test_df.document.isna().sum()

3

In [17]:
# Null 데이터 제거
test_df.dropna(how='any', inplace=True)
test_df.shape

(49997, 3)

In [18]:
# 중복 확인
test_df.document.nunique()

49157

In [19]:
# 중복 제거
test_df.drop_duplicates(subset=['document'], inplace=True)
test_df.shape

(49157, 3)

In [20]:
# Label 분포
test_df.label.value_counts()

1    24711
0    24446
Name: label, dtype: int64

### 2. 텍스트 전처리

- train dataset

In [22]:
# 한글 이외의 문자는 공백으로 처리하고 strip
train_df.document = train_df.document.str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣]',' ').str.strip()
train_df.head(3)

,id,document,label
0,9976970,아 더빙 진짜 짜증나네요 목소리,0
1,3819312,흠 포스터보고 초딩영화줄 오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0


In [23]:
review = 'What a good movie!!!'
review1 = '                    '
review1.strip()

''

In [24]:
# '' 만 남은 데이터 --> np.nan 으로 변경시킨 후 제거
train_df.document.replace('', np.nan, inplace=True)
train_df.document.isna().sum()

789

In [25]:
# Null 데이터 제거
train_df.dropna(how='any', inplace=True)
train_df.shape

(145393, 3)

- test dataset

In [26]:
test_df.document = test_df.document.str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣]',' ').str.strip()
test_df.document.replace('', np.nan, inplace=True)
test_df.dropna(how='any', inplace=True)
test_df.shape

(48852, 3)

- 전처리가 끝난 데이터는 저장

In [27]:
train_df.to_csv('data/naver_movie_train_전처리완료.csv', sep='\t', index=False)
test_df.to_csv('data/naver_movie_test_전처리완료.csv', sep='\t', index=False)

### 3. 한글 처리 - SOYNLP

In [28]:
import joblib
scores = joblib.load('data/scores.pkl')

In [29]:
from soynlp.tokenizer import MaxScoreTokenizer
max_tokenizer = MaxScoreTokenizer(scores)

In [30]:
# 형태소로 tokenize
text = '교도소 이야기구먼 솔직히 재미는 없다평점 조정'
max_tokenizer.tokenize(text)

['교도소', '이야기', '구먼', '솔직', '히', '재미는', '없다', '평점', '조정']